In [5]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter
import matplotlib.cm as cm
import cv2
from __future__ import print_function
from pathlib import Path
import shutil


# Define path variables

In [3]:
dataset_path = Path("/mnt/d/ShapeNetCore.v2/ShapeNetCore.v2")
current_path = Path("./")
image_path = current_path / "dataset"
image_path.mkdir(parents=True, exist_ok=True)
data_path = current_path / "outline"
data_path.mkdir(parents=True, exist_ok=True)
sample_path =  current_path / "sampled"
sample_path.mkdir(parents=True, exist_ok=True)
mask_path_highres = current_path / "mask_highres"
mask_path_highres.mkdir(parents=True, exist_ok=True)
mask_path = current_path / "mask"
mask_path.mkdir(parents=True, exist_ok=True)

# Gather 2D images from ShapeNet base folder

In [ ]:
for img in dataset_path.rglob("screenshots/*.png"):
    image_name = img.name
    obj_cls = img.parent.parent.parent.name
    obj_id = img.parent.parent.name
    print(f"cls : {obj_cls} - id : {obj_id} - image_name : {image_name}")
    img_folder = (image_path / obj_cls)
    img_folder.mkdir(parents=True, exist_ok=True)
    shutil.copy(img, img_folder / image_name)

# Create object masks

In [8]:
for path in image_path.rglob("*.png"):
    # Read image
    im_in = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)

    # Threshold.
    th, im_th = cv2.threshold(im_in, 254, 255, cv2.THRESH_BINARY)
    im_th_inv=cv2.bitwise_not(im_th)

    # # Copy the thresholded image.
    im_floodfill = im_th.copy()

    # # Mask used to flood filling.
    # # Notice the size needs to be 2 pixels than the image.
    h, w = im_th.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)

    # # Floodfill from point (0, 0)
    cv2.floodFill(im_floodfill, mask, (0,0), 0)
    # # Combine the two images to get the foreground.
    im_out = im_th_inv | im_floodfill

    save_dir = mask_path_highres / path.parent.name
    save_dir.mkdir(parents=True, exist_ok=True)
    
    cv2.imwrite(str(save_dir / path.name), im_out)

# Resize and crop to object

In [6]:
resize = (100,100)
antialias_threshold = 30

In [12]:
for path in mask_path_highres.rglob("*.png"):
    mask = np.array(Image.open(path))
    
    width , height  = mask.shape
    
    left = np.argmax(np.argmax(mask, axis=0) > 0)
    top = np.argmax(np.argmax(mask, axis=1) > 0)
    right = width - np.argmax(np.argmax(np.flip(mask), axis=0) > 0)
    bottom = height - np.argmax(np.argmax(np.flip(mask), axis=1) > 0)
    
    height, width = bottom - top, right - left
    side = max(height, width)
        
    mask = mask[top:top+height, left:left+width]
    mask = np.pad(mask, (((side-height) // 2, (side-height) // 2), 
                         ((side-width) // 2, (side-width) // 2)))
    
    mask = Image.fromarray(mask).resize(resize, 3)
    mask = np.array(mask, dtype=np.uint8) > antialias_threshold
        
    mask = Image.fromarray(mask)
        
    save_dir = mask_path / path.parent.name
    save_dir.mkdir(parents=True, exist_ok=True)
    

    mask.save(save_dir / path.name)

# Get the outline from masks

In [13]:
for path in mask_path.rglob("*.png"):
    img = Image.open(path).convert("L")
    img = img.filter(ImageFilter.FIND_EDGES)
    
    save_dir = data_path / path.parent.name
    save_dir.mkdir(parents=True, exist_ok=True)
    
    img.save(save_dir / path.name)

# Sample outline points for the input representation

In [25]:
n_sample = 100
n_from_each = 1

In [26]:
import scipy, threading

In [ ]:
for path in data_path.rglob("*.png"):
    img = Image.open(path)
    img = np.array(img, dtype=np.uint8)
    
    n = np.count_nonzero(img)

    sparse_img = scipy.sparse.coo_matrix(np.array(img))
    
    def get_sample(sparse_img, n, i):
    
        samples = np.random.choice(np.arange(0, n), n_sample, False)

        sparse_img = scipy.sparse.coo_matrix((sparse_img.data[samples], 
                                          (sparse_img.row[samples], sparse_img.col[samples])), 
                                         (100, 100))
        img = sparse_img.todense()
        img = Image.fromarray(img)


        save_dir = sample_path2 / path.parent.name
        save_dir.mkdir(parents=True, exist_ok=True)
        img.save(save_dir / (f"{i}_" + path.name))
    
    threads = []
    for i in range(n_from_each):
        t = threading.Thread(target=get_sample, args=(sparse_img, n, i))
        threads.append(t)
        t.start()
    for t in threads:
        t.join()